In [1]:
import os

from dotenv import load_dotenv, find_dotenv

from langchain_community.document_loaders import PyPDFLoader 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

In [2]:
load_dotenv(find_dotenv())

DB_FAISS_PATH = "vectorstore/db_faiss"

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

# Langsmith

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "AI-COURSE-RAG"

In [15]:
# Loading docs 

loader = PyPDFLoader("D:\AI COURSE\GEN AI\DAY - 26\data\SYSTEM DESIGN PDF.pdf")
loader.load()

[Document(metadata={'producer': 'Skia/PDF m134', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/134.0.0.0 Safari/537.36', 'creationdate': '2025-06-30T07:44:23+00:00', 'title': 'SYSTEM DESIGN', 'moddate': '2025-06-30T07:44:23+00:00', 'source': 'D:\\AI COURSE\\GEN AI\\DAY - 26\\data\\SYSTEM DESIGN PDF.pdf', 'total_pages': 55, 'page': 0, 'page_label': '1'}, page_content='SYSTEM DESIGN \nMore resources : \nhttps://github.com/InterviewReady/system-design-resources\nModule 1: Basics\nWhat is System Design?\nSystem design is the process of defining the architecture, components, \nmodules, interfaces, and data flow of a system to meet specific requirements. It \ninvolves creating a blueprint for building scalable, reliable, and efficient \nsoftware systems. System design bridges the gap between requirements (what \nthe system should do) and implementation (how it will be built). It encompasses \nboth high-level design (overall architecture) an

In [16]:
docs = loader.load()

In [17]:
# Text Splitter 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
docs = text_splitter.split_documents(docs)

In [22]:
docs[3]

Document(metadata={'producer': 'Skia/PDF m134', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/134.0.0.0 Safari/537.36', 'creationdate': '2025-06-30T07:44:23+00:00', 'title': 'SYSTEM DESIGN', 'moddate': '2025-06-30T07:44:23+00:00', 'source': 'D:\\AI COURSE\\GEN AI\\DAY - 26\\data\\SYSTEM DESIGN PDF.pdf', 'total_pages': 55, 'page': 1, 'page_label': '2'}, page_content='RAM, etc.).\nExplanation\ue092 You increase the power of a single machine to handle \nmore load. For example, upgrading a serverʼs RAM from 16GB to 64GB.\nPros:\nSimpler to implement (no need for distributed systems).\nLess overhead for synchronization.\nCons:\nLimited by hardware constraints (e.g., max CPU/RAM capacity).\nSingle point of failure: If the server fails, the system goes down.\nExample\ue092 Upgrading a database serverʼs storage to handle larger \ndatasets.\nSYSTEM DESIGN\n2')

In [23]:
# Embedding 
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={"device": "cpu"})

In [7]:
# Vector store 

vectorstore = FAISS.from_documents(docs,embeddings)
FAISS.save_local(vectorstore,"vectorstore_new/db_faiss")

In [24]:
# LLM 

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",temperature=0.2)


In [25]:
# Prompt 


prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant. Answer the user's question based on the context provided and the chat history. If you don't know the answer, just say that you don't know. Don't try to make up an answer.\n\nContext: {context}"),
        ("human", "{input}"),
    ])

In [10]:
# Creating stuff doc chain 

stuff_doc_chain = create_stuff_documents_chain(llm,prompt)

In [12]:
# Loading vector store 

vectorstore = FAISS.load_local("vectorstore_new/db_faiss",embeddings,allow_dangerous_deserialization=True)

# Creating retriever 

retriever = vectorstore.as_retriever()

In [13]:
# Creating retrieval chain 

retrieval_chain = create_retrieval_chain(retriever,stuff_doc_chain)

In [26]:
# Invoking the chain 

response = retrieval_chain.invoke({"input": "What is consistent hashing?"})

response["answer"]

'Consistent hashing is a technique used to distribute data across a cluster of nodes in a way that minimizes data movement when nodes are added or removed. Nodes and data are mapped to a hash ring using a hash function. Each data item is assigned to the nearest node on the ring (clockwise). When a node is added or removed, only a small subset of data is reassigned, unlike traditional hashing.'